<a href="https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_stage_2_verify_new_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""ali_build_model_stage_2_MobileNetV2_aug_1000_dense.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wSVEmcoWzRm-7B6UGkHB8ysfMDRT_xK5
"""

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
src_data=  'drive/My Drive/geo/stage2/stage2_new_test/'
src =  'drive/My Drive/geo/stage2/'
src_model=  'drive/My Drive/geo/'

In [4]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, \
Conv2DTranspose,BatchNormalization
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers

Using TensorFlow backend.


In [5]:
'''
src_y = 'drive/My Drive/geo/stage2/stage2_new_test/stage2_np/y/'
y_list =  os.listdir(src_y)
y_list.sort()

f_list = []

for _y in y_list:
  temp = np.load(src_y+_y)
  f_list.append(temp)
  
                     


f = np.stack(f_list, axis=0)
f.shape


import pickle 
with open (src+'y.npy', 'wb') as handle:
  pickle.dump(f, handle)


src_x = 'drive/My Drive/geo/stage2/stage2_new_test/stage2_np/x/'
x_list =  os.listdir(src_x)
x_list.sort()
f_list_x = []

for _x in x_list:
  temp = np.load(src_x+_x)
  f_list_x.append(temp)
  
                     


f_x = np.stack(f_list_x, axis=0)
f_x.shape


import pickle 
with open (src+'x.npy', 'wb') as handle:
  pickle.dump(f_x, handle)

'''

"\nsrc_y = 'drive/My Drive/geo/stage2/stage2_new_test/stage2_np/y/'\ny_list =  os.listdir(src_y)\ny_list.sort()\n\nf_list = []\n\nfor _y in y_list:\n  temp = np.load(src_y+_y)\n  f_list.append(temp)\n  \n                     \n\n\nf = np.stack(f_list, axis=0)\nf.shape\n\n\nimport pickle \nwith open (src+'y.npy', 'wb') as handle:\n  pickle.dump(f, handle)\n\n\nsrc_x = 'drive/My Drive/geo/stage2/stage2_new_test/stage2_np/x/'\nx_list =  os.listdir(src_x)\nx_list.sort()\nf_list_x = []\n\nfor _x in x_list:\n  temp = np.load(src_x+_x)\n  f_list_x.append(temp)\n  \n                     \n\n\nf_x = np.stack(f_list_x, axis=0)\nf_x.shape\n\n\nimport pickle \nwith open (src+'x.npy', 'wb') as handle:\n  pickle.dump(f_x, handle)\n\n"

In [7]:
data= np.load(src_data+'x_test.npy')
label = np.load(src_data+'y_test.npy')
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)



(6000, 21, 21, 20)
(6000, 20, 20, 1)


# Reduce H,W from 21 to 11, Reduce channel from 21 to 11





In [0]:
data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

In [9]:
train_data.shape

(6000, 21, 21, 20)

In [0]:
model = load_model(src_model+'stage2_reproduce_water_first_try_2_prepro_ch32.h5')

# Testing Set NRMSE Loss

In [0]:

def nrmse(y_true,y_pred):

    return (np.sqrt(np.mean(np.square(y_true - y_pred))))/np.mean(y_pred)

In [16]:







prediction = model.predict(train_data)

print (prediction.shape)

prediction = np.exp(prediction)


#pred = prediction.reshape(1800,-1)
#print (pred.shape)

print (label.shape)



print ('test set NRMSE error', nrmse(label,prediction))

(6000, 20, 20, 1)
(6000, 20, 20, 1)
test set NRMSE error 0.20334222319299025
